## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
    df.drop(columns=["inspector_name,violator_name,violation_street_number"],inplace=True)


train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    import datetime as dt
    import matplotlib.pyplot as plt
    import seaborn as sns
    df = pd.read_csv("train.csv",index_col="ticket_id", encoding = "ISO-8859-1") 
    #df.set_index("ticket_id",inplace=True)

    return df # Your answer here
df = blight_model()
df.describe()

c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,violation_street_number,violation_zip_code,mailing_address_str_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance
count,2.503060e+05,0.0,2.467040e+05,250305.000000,250306.000000,250306.000000,250306.000000,250306.000000,250306.0,250306.000000,250306.000000,250306.000000,159880.000000
mean,1.064986e+04,NaN,9.149788e+03,374.423435,12.774764,6.387382,21.494506,0.125167,0.0,268.685356,48.898986,222.449058,0.072536
std,3.188733e+04,NaN,3.602034e+04,707.195807,9.607344,4.803672,56.464263,3.430178,0.0,626.915212,222.422425,606.394010,0.259374
min,0.000000e+00,NaN,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-7750.000000,0.000000
25%,4.739000e+03,NaN,5.440000e+02,200.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,1.024400e+04,NaN,2.456000e+03,250.000000,20.000000,10.000000,10.000000,0.000000,0.0,140.000000,0.000000,25.000000,0.000000
75%,1.576000e+04,NaN,1.292725e+04,250.000000,20.000000,10.000000,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
max,1.415411e+07,NaN,5.111345e+06,10000.000000,20.000000,10.000000,1000.000000,350.000000,0.0,11030.000000,11075.000000,11030.000000,1.000000


## Exploratory Data Analysis

In [3]:
df1= df[ ["violation_street_name","ticket_issued_date","hearing_date","disposition",
          "fine_amount","discount_amount",
          "judgment_amount", "compliance"]]

# recover as many violation_street_name as possible from comparing with census and lonlan dataset.
#create two data sets one toe explore data the other one without compliance detail and drop data only on this one and after recovering street name
print("initial number of tickets: {}".format(len(df1)))
print("after droping na values: {}".format(len(df1.dropna())))
df1.dropna(inplace=True)

initial number of tickets: 250306
after droping na values: 159653


c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
# time interval between hearing and ticket issued date

df1["ticket_issued_date"] = pd.to_datetime(df1["ticket_issued_date"])# similar-> df1["ticket_issued_date"].astype("datetime64[D]")
df1["hearing_date"]=pd.to_datetime(df1["hearing_date"])
#df1["interval_hearing_ticket"] = df1.hearing_date - df1.ticket_issued_date
#df1["interval_hearing_ticket"] = df1["interval_hearing_ticket"].apply(lambda x: x.days)
#df1.drop(df1[ df1.interval_hearing_ticket<0].index, inplace=True)#makes no sense when hearing date before ticket_issued_date
df1.drop(columns=["hearing_date","ticket_issued_date"],inplace=True)

c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\pandas\core\frame.py:3940: Setting

In [11]:
df1["disposition"] = df1["disposition"].astype("category")
df1_dummy = pd.get_dummies(df1,prefix="disposition",columns=["disposition"])
#list(df1["disposition"].unique())
df1_dummy.head(5)

c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,violation_street_name,fine_amount,discount_amount,judgment_amount,compliance,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination
ticket_id,,,,,,,,,
22056,TYLER,250.0,0.0,305.0,0.0,0,0,1,0
27586,CENTRAL,750.0,0.0,855.0,1.0,0,0,0,1
22046,NORTHFIELD,250.0,0.0,305.0,0.0,0,0,1,0
18738,BRENTWOOD,750.0,0.0,855.0,0.0,0,0,1,0
18735,MT ELLIOTT,100.0,0.0,140.0,0.0,0,0,1,0


In [ ]:
# encoding non-numerical variables
encoding_lists =[]
non_numerical_vars = ["disposition","violation_street_name"]
for i in non_numerical_vars:
    encoding_list = list(enumerate( df1[i].unique() ))
    encoding_lists.append(  encoding_list  )
    df1[i] = df1[i].apply(  lambda x: [y[0] for y in encoding_list if y[1]==x][0]  )

In [ ]:
##      # mapping disposition categories (to num labels) 
##      disposition_lookup = list(enumerate(df1.disposition.unique()))
##      lookup_func = lambda x: [y[0] for y in disposition_lookup if y[1]==x][0]
##      df1["disposition"] = df1["disposition"].apply( lookup_func ) 
##      # mapping compliance_detail
##      compliance_detail_lookup = list(enumerate(df1.compliance_detail.unique()))
##      lookup_func_cd = lambda x: [y[0] for y in compliance_detail_lookup if y[1]==x][0]
##      df1["compliance_detail"] = df1["compliance_detail"].apply( lookup_func_cd ) 
##      # mapping street name 
##      violation_street_name_lookup = list(enumerate(df1.violation_street_name.unique()))
##      lookup_func_vs = lambda x: [y[0] for y in violation_street_name_lookup if y[1]==x][0]
##      df1["violation_street_name"] = df1["violation_street_name"].apply( lookup_func_vs ) 
##      #categorical functions
##      #for i in ["violation_street_name","compliance","compliance_detail"]:
##      #    df1[i]=df1[i].astype("category")
##      df1["compliance"] = df1["compliance"].astype("str").astype("category")

In [ ]:
df1.head(5)

In [ ]:
print(df1.dtypes)
df1.describe()

In [13]:
#vars_to_plot = ["fine_amount","discount_amount","judgment_amount","interval_hearing_ticket","compliance_detail","disposition"]
#vars_for_model = ["fine_amount","discount_amount","judgment_amount","interval_hearing_ticket","violation_street_name","disposition"]
#vars_for_model = ["fine_amount","discount_amount","judgment_amount","violation_street_name","disposition"]
#vars_for_model = ["fine_amount","discount_amount","judgment_amount","violation_street_name","disposition"]
vars_for_model = ["fine_amount","discount_amount","judgment_amount","disposition"]

In [ ]:
import seaborn as sns
%matplotlib notebook
sns.set(style="ticks")
sns.pairplot(df1,vars=vars_for_model,diag_kind="hist",hue="compliance")

In [ ]:
#sns.kdeplot(df1.interval_hearing_ticket)
#plt.gca().set_yscale('log')
sns.catplot(x="compliance", kind="count", edgecolor=".6", data=df1);

## street names and lat-log indexed  by ticket_id

In [ ]:
dfa = pd.read_csv("addresses.csv")
dfa.dropna(inplace=True)
print(len(dfa))
street_parser = lambda x:  x.split(",")[0].split(" ")[1].strip()
address_parser = lambda x: x.split(",")[0].strip()
dfa["street"] = dfa["address"].apply(street_parser)
dfa["address"] = dfa["address"].apply(address_parser)
dfa.head(3)
#ctiy var not usea, unique value is Detroit
#city_parser   = lambda x : x.split(",")[1].strip().split(" ")[0]
#dfa["city"]   = dfa["address"].apply( city_parser )
#len(dfa.street.unique())

In [ ]:
dflalo = pd.read_csv("latlons.csv")
dflalo.dropna(inplace=True)
dflalo["street"]=dflalo["address"].apply(street_parser)
dflalo["address"]=dflalo["address"].apply(address_parser)
print(len(dflalo) )
dflalo.head(3)

In [ ]:
print(  "Number of different street names in adresses csv: {}".format(len(dfa.street.unique())) )
print(  "Number of different street names in latlon  csv: {}".format(len(dflalo.street.unique())) )
print(  "Number of different street names in train data set: {}".format(len(df1.violation_street_name.unique())) )

In [ ]:
df_spatial = dfa.set_index("address").join(dflalo.set_index("address"),how="left",lsuffix='_caller', rsuffix='_other')
df_spatial.dropna(inplace=True)
len(df_spatial)

## Model Creation

In [ ]:
#model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

#models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

#metric chosen AUC
from sklearn.metrics import roc_curve, auc, roc_auc_score

### Simplest 0:  Logistic classifier  1-cv fold 

In [ ]:
#vars_for_model.append("compliance")
print(vars_for_model)
#df1.columns

In [ ]:
X = df1[vars_for_model].values #as_matrix();
y = df1["compliance"].values    #.as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

print("y_1/y0 full data: {}".format(sum(y==1)/sum(y==0)))
print("y_1/y0 train    : {}".format(sum(y_train==1)/sum(y_train==0)))
print("y_1/y0 test     : {}".format(sum(y_test==1)/sum(y_test==0)))
#Conclusion: scikit learn handels and homogeneous distribution of the data

scaler=MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

logic_reg = LogisticRegression()

In [ ]:
#np.shape(X_train)
#print(X_train[:1])
#print(X_train_scaled[:1])

In [ ]:
grid_values = {'C': [0.01, 0.1, 1, 10, 100, 150, 200], 'penalty':['l1','l2']}
grid_clf = GridSearchCV(logic_reg,param_grid=grid_values,scoring='roc_auc',n_jobs=-1)

# alternative metric to optimize over grid parameters: AUC
grid_clf.fit(X_train_scaled, y_train)
y_decision_fn_scores_auc = grid_clf.decision_function(X_test_scaled) 

print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
print('Grid best score (AUC): ', grid_clf.best_score_)

In [ ]:
y_pred= grid_clf.predict(X_test_scaled)
print(grid_clf.best_params_)
print(classification_report(y_test,y_pred))

# Test data

In [14]:
################### prepare test set ################################
df_cv = pd.read_csv("test.csv",index_col="ticket_id", encoding = "ISO-8859-1") 
#df1_cv = df[ ["violation_street_name","disposition","fine_amount","discount_amount", "judgment_amount"]]
                    #"ticket_issued_date","hearing_date",
df1_cv = df[vars_for_model]
    # time interval between hearing and ticket issued date
    #df1_cv["ticket_issued_date"] = pd.to_datetime(df1_cv["ticket_issued_date"])
    #df1_cv["hearing_date"]=pd.to_datetime(df1_cv["hearing_date"])
    #df1_cv["interval_hearing_ticket"] = df1_cv.hearing_date - df1_cv.ticket_issued_date
    #df1_cv["interval_hearing_ticket"] = df1_cv["interval_hearing_ticket"].apply(lambda x: x.days)
    #df1_test.drop(columns=["hearing_date","ticket_issued_date"],inplace=True)

In [31]:
df1_cv["disposition"] = df1_cv["disposition"].astype("category")
df1_cv_dummy = pd.get_dummies(df1_cv,prefix="disposition",columns=["disposition"])
df1_cv_dummy.head(5)

df1.columns.tolist().remove("compliance")

c:\users\c.lizarazosabogal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
print("############# columns df train: ")
for i in df1_dummy.columns.to_list(): print(i)
print("############# columns df test: ")
for i in df1_cv_dummy.columns.to_list():print(i)
#df1_cv[df1_dummy.columns]
df1_cv_dummy[ df1_dummy.columns.to_list() ]

############# columns df train: 
violation_street_name
fine_amount
discount_amount
judgment_amount
compliance
disposition_Responsible (Fine Waived) by Deter
disposition_Responsible by Admission
disposition_Responsible by Default
disposition_Responsible by Determination
############# columns df test: 
fine_amount
discount_amount
judgment_amount
disposition_Not responsible by City Dismissal
disposition_Not responsible by Determination
disposition_Not responsible by Dismissal
disposition_PENDING JUDGMENT
disposition_Responsible (Fine Waived) by Deter
disposition_Responsible by Admission
disposition_Responsible by Default
disposition_Responsible by Determination
disposition_SET-ASIDE (PENDING JUDGMENT)


In [27]:
df1_cv_dummy[ df1_dummy.columns.to_list() ]

KeyError: "['violation_street_name', 'compliance'] not in index"

In [ ]:
    # encoding non-numerical variables
encoding_lists_cv =[]
#non_numerical_vars = ["violation_street_name","disposition"]
non_numerical_vars = ["disposition"]
for i in non_numerical_vars:
    encoding_list = list(enumerate( df1_cv[i].unique() ))
    encoding_lists_cv.append(  encoding_list  )
    df1_cv[i] = df1_cv[i].apply(  lambda x: [y[0] for y in encoding_list if y[1]==x][0]  )
###################################################################
X_cv = df1_cv[vars_for_model].values 
#scaler=MinMaxScaler().fit(X_cv)
#X_cv_scaled = scaler.transform(X_cv)
#y_cv_prob   = grid_clf.predict_proba(X_cv_scaled)
#y_cv_prob   = y_cv_prob[:,1]
    

In [ ]:
for i in non_numerical_vars:
    print("Trainset unique elements {}: {}".format(i,len(list(df1[i].unique()))))
    print("Testset  unique elements {}: {}".format(i,len(list(df1_cv[i].unique()))))
    

In [ ]:
print("##### disposition values for df1_cv")
for i in encoding_lists_cv[0]: print(i)
print("#####  disposition values for df1")
for i in encoding_lists[0]: print(i)

### figuring out nas  of each column of test data, and filling them:

In [ ]:

len_df_cv = len(df1_cv)
for i in df1_cv.columns:
    print(i)
    nas_var_i = df1_cv[i].isna().sum()
    print("{}, {}%".format(nas_var_i, int(nas_var_i/len_df_cv*100)))

In [ ]:
df1_cv[ df1_cv["fine_amount"].isnull()==True  ]

In [ ]:
df1_cv["fine_amount"].fillna(0)
df1_cv.loc[105992]

### Playing with get_dummies method

In [ ]:
df_toy = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'],'C': [1, 2, 3]})
df_toy

In [ ]:
pd.get_dummies(df, prefix=['col1', 'col2'])

### getting dummies for "disposition"

In [ ]:
df1["disposition"] = df1["disposition"].astype("category")
df1.dtypes

In [ ]:
### for training data
df1_dummy = pd.get_dummies(df1,prefix="disposition")
df1_dummy.head(5)

In [ ]:
encoding_lists_cv[0]